# PDE-FIND for the Kuramoto Sivashinsky Equation

This notebook demonstrates PDE-FIND on the Kuramoto Sivashinsky equation.
$$
u_t + u_{xxxx} + uu_x + u_{xx} = 0
$$

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings("ignore")

import numpy as np
from numpy.random import default_rng
import pandas as pd

import sys; sys.path.append('../')
from PDE_FIND import *

import sys; sys.path.append('../../parametric-discovery/')
from best_subset import *
from frols import frols
from p_linear_regression import PLinearRegression
from r_pca import R_pca
from pde_diff_new import RPCA
from RobustPCA.rpca import RobustPCA

import scipy.io as sio
from scipy.signal import savgol_filter, butter, filtfilt, wiener
from scipy import integrate
from scipy.integrate import simpson, trapz, romb
import itertools

from sklearn.preprocessing import normalize
from sklearn.linear_model import Ridge, BayesianRidge, LinearRegression as SkLinearRegression
from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from abess.linear import LinearRegression
import pysindy as ps

from os.path import join as join_path
FIGURE_EXPORT_PATH = "/Users/pongpisit/Documents/figure_export/"

def evaluate_coefficients(prediected_coeffs):
    GROUND = np.array([0.1, -1])
    errs = 100*np.abs(GROUND-np.array(prediected_coeffs).flatten())/np.abs(GROUND)
    return errs.mean(), errs.std()

import torch, sympytorch
import torch.nn as nn
import torch.nn.functional as F
from sympy import symbols, simplify, lambdify
from mathparser import math_eval
from varname import nameof
from misc import h5file

import derivative
from tvregdiff import TVRegDiff, tvregdiff, numdiff, pysindydiff, savgol_denoise
from functools import partial
from findiff import FinDiff

from tqdm import tqdm, trange
from tsmoothie.smoother import *

Sklearn's version: 1.2.2
mrmr is not installed in the env you are using. This may cause an error in future if you try to use the (missing) lib.


In [2]:
data = sio.loadmat('../Datasets/kuramoto_sivishinky.mat')
u = data['uu']
x = data['x'][:,0]
t = data['tt'][0,:]
dt = t[1]-t[0]
dx = x[2]-x[1]
X, T = np.meshgrid(x, t)
XT = np.asarray([X, T]).T

assert np.max(x)/32 == np.pi

In [3]:
np.random.seed(0)
noise_lv = 30
un = u + 0.01*noise_lv*u.std()*np.random.randn(u.shape[0],u.shape[1])

In [4]:
# un = np.load("./denoised_u_files/KS_noise30_reduced_dctksvdreg.npy")

# un = np.load("./denoised_u_files/KS_noise30_reduced_dctV2ksvdreg_21x21.npy")
un = np.load("./denoised_u_files/KS_noise30_reduced_dctV2ksvdreg_25x25.npy")

In [5]:
library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]
pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=4,
    spatial_grid=x,
    is_uniform=True,
)

print('STLSQ model: ')
optimizer = ps.STLSQ(threshold=0.1, alpha=1e-5, normalize_columns=False)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
model.fit(np.expand_dims(u, -1), t=dt)
model.print()

STLSQ model: 
(x0)' = -0.994 x0_11 + -0.996 x0_1111 + -0.992 x0x0_1


In [6]:
class Differentiator(ps.BaseDifferentiation):
    def __init__(self, diff_name, diff_func_kwargs, 
                 d=1, axis=1, 
                 is_uniform=True, periodic=False):
        super(Differentiator, self).__init__()
        self.diff_name = diff_name
        self.diff_func_kwargs = diff_func_kwargs
        self.diff_func = getattr(derivative, self.diff_name)(**diff_func_kwargs)
        
        self.d = d
        self.diff = partial(pysindydiff, **{"diff_method":self.diff_func, "order":self.d})
        
        # Other info...
        self.axis = axis
        self.is_uniform = is_uniform
        self.periodic = periodic
        self.transform = np.vectorize(composite_function(self.diff, lambda _: _, left2right=True), signature="(m),(m)->(m)")

    def _differentiate(self, x, t):
        in_shape = x.shape
        if len(in_shape) == 2: x = np.expand_dims(x, -1) # x should now be 3-dimensional
        if isinstance(t, float) and self.is_uniform: 
            t = np.linspace(0, stop=t*(x.shape[self.axis]-1), num=x.shape[self.axis])
        out = []
        # wrt to x var
        if self.axis == 0:
            for i in range(x.shape[-1]):
                diff = self.transform(x[:,:,i].T, t).T
                out.append(np.expand_dims(diff, axis=-1))
        # wrt to time var
        elif self.axis == 1:
            for i in range(x.shape[-1]):
                diff = self.transform(x[:,:,i], t)
                out.append(np.expand_dims(diff, axis=-1))
        return np.concatenate(out, axis=-1).reshape(in_shape)

class FiniteDifferentiator(ps.BaseDifferentiation):
    def __init__(self, acc=2, d=1, axis=1, is_uniform=True, periodic=False):
        super(FiniteDifferentiator, self).__init__()        
        self.acc = 2*(acc//2)
        self.d = d
        self.axis = axis
        self.is_uniform = is_uniform
        self.periodic = periodic
    def _differentiate(self, x, t):
        if self.is_uniform:
            if isinstance(t, float): 
                dt = t
            else: 
                dt = t[1]-t[0]
            self.diff = FinDiff(self.axis, dt, self.d, acc=self.acc)
        else:
            raise NotImplementedError("is_uniform=False")
            self.diff = FinDiff(self.axis, self.d, acc=self.acc)
        return self.diff(x)

In [7]:
### originally from derivative ###
## slow ##
# diff_name, diff_func_kwargs = "Kalman", {'alpha':kalpha}
## fast ##
# diff_name, diff_func_kwargs = "FiniteDifference", {'k':1}
# differentiation_method = Differentiator
# differentiation_kwargs = {"diff_name": diff_name, "diff_func_kwargs": diff_func_kwargs}

# originally from pysindy
# differentiation_method, differentiation_kwargs = ps.SmoothedFiniteDifference, {}

# originally from findiff (accurate and fast)
differentiation_method, differentiation_kwargs = FiniteDifferentiator, {'acc':2}

In [8]:
diff_order = 4
weak_pde_lib = ps.WeakPDELibrary(library_functions=[lambda x: x, lambda x: x * x], 
                                 function_names=[lambda x: x, lambda x: x + x], 
                                 derivative_order=diff_order, p=diff_order, 
                                 spatiotemporal_grid=XT, 
                                 include_bias=False, is_uniform=True, K=10000, # new random K points in every calls to the ps.WeakPDELibrary
                                 differentiation_method=differentiation_method, 
                                 differentiation_kwargs=differentiation_kwargs, 
                                 cache=True
                                )
kwargs = {'fit_intercept':False, 'copy_X':True, 'normalize_columns':False}

# X_pre = weak_pde_lib.fit_transform(est_u_sol).reshape(-1, len(weak_pde_lib.get_feature_names()))
# y_pre = weak_pde_lib.convert_u_dot_integral(est_u_sol)
# X_pre, y_pre, fns = ps_features(un, t, weak_pde_lib, kwargs)

In [9]:
optimizer = L0BNB(max_nonzeros=4, lam=5e-2, is_normal=True, normalize_columns=False, threshold=1e-4) # tune lam จาก (X_pre, y_pre) | ตอนนี้ยังไม่ได้ tune
# optimizer = BruteForceRegressor(3)
# if feature_library=weak_pde_lib, then just differentiation_method=None is fine.
n_ensemble_models = 50
model = ps.SINDy(feature_library=weak_pde_lib, optimizer=optimizer, 
                 differentiation_method=differentiation_method(**differentiation_kwargs), 
                 cache=True,
                 feature_names=['u'])

In [10]:
model.fit(np.expand_dims(un, -1), t=dt, ensemble=True, library_ensemble=True, n_candidates_to_drop=1, n_models=n_ensemble_models)
print(model.get_feature_names())

['u', 'uu', 'u_1', 'u_11', 'u_111', 'u_1111', 'uu_1', 'uuu_1', 'uu_11', 'uuu_11', 'uu_111', 'uuu_111', 'uu_1111', 'uuu_1111']


In [11]:
X_pre, y_pre = np.squeeze(model.feature_library.cached_xp_full), model.cached_x_dot
print_pde(model.get_coef_list()[np.argmin(np.sum((np.squeeze(np.tensordot(X_pre, np.array(model.get_coef_list()).T, axes=([-1], [0])), axis=1)-y_pre)**2, axis=0))].reshape(-1,1), model.get_feature_names())

u_t = (-0.991215 +0.000000i)u_11
    + (-0.987916 +0.000000i)u_1111
    + (-0.998875 +0.000000i)uu_1
   


In [12]:
model_results = {}
for effective_indices in set([tuple(np.nonzero(model.get_coef_list()[i][0])[0]) 
                              for i in range(len(model.get_coef_list()))]):
    coeff = np.linalg.lstsq(X_pre[:, effective_indices], y_pre, rcond=None)[0]
    mse = ((y_pre-X_pre[:, effective_indices]@coeff)**2).mean()
    n_terms = len(effective_indices)
    if (n_terms > 0) and (n_terms not in model_results or mse < model_results[n_terms][1]):
        model_results[n_terms] = effective_indices, mse, coeff
        
for com in model_results:
    effective_indices, _, coeff = model_results[com]
    print_pde(coeff, np.array(model.get_feature_names())[list(effective_indices)])
    
# model_results

u_t = (-0.993022 +0.000000i)u_11
    + (-0.995778 +0.000000i)u_1111
    + (-0.998724 +0.000000i)uu_1
   
u_t = (-0.995368 +0.000000i)u_11
    + (-0.996953 +0.000000i)u_1111
    + (-0.998871 +0.000000i)uu_1
    + (0.000392 +0.000000i)uuu_11
   
u_t = (-0.577216 +0.000000i)uu_1
    + (-0.049968 +0.000000i)uuu_11
   


In [13]:
# optimizer = L0BNB(max_nonzeros=5, lam=1e-1, is_normal=False, normalize_columns=False, threshold=1e-4)
# optimizer.fit(X_pre, y_pre).coef_

In [14]:
mses = []; optimal_coeff = None
all_possible_models = list(itertools.combinations(range(14), 3))
for effective_indices in all_possible_models:
    coeff = np.linalg.lstsq(X_pre[:, effective_indices], y_pre, rcond=None)[0]
    mses.append(((y_pre-X_pre[:, effective_indices]@coeff)**2).mean())
optimal_coeff = np.linalg.lstsq(X_pre[:, all_possible_models[np.argmin(mses)]], 
                                y_pre, rcond=None)[0]
all_possible_models[np.argmin(mses)], np.min(mses), optimal_coeff

((3, 5, 6),
 0.015537920645042988,
 array([[-0.99302186],
        [-0.99577807],
        [-0.99872439]]))

In [15]:
(np.abs(optimal_coeff+1)*100).mean()

0.4158557680351338

In [16]:
# com = 3
baye_alpha = 1e-6
for com in sorted(model_results.keys()):
    brr = BayesianRidge(compute_score=True, alpha_1=baye_alpha, alpha_2=baye_alpha)
    brr.fit(X_pre[:, model_results[com][0]], y_pre)
    print(f"com = {com},", np.trace(np.sqrt(brr.sigma_)))

com = 2, 0.005501994
com = 3, 0.0035781963
com = 4, 0.004132747


#### WAIC from GPT (need further checking)

In [17]:
# # WAIC from GPT
# com = 3

# # Fit Bayesian Ridge Regression model
# regressor = BayesianRidge()
# X = X_pre[:, model_results[com][0]]
# y = y_pre
# regressor.fit(X, y)

# # Obtain posterior samples
# posterior_samples = np.random.multivariate_normal(regressor.coef_, regressor.sigma_, size=1000)

# # Compute log-likelihood
# log_likelihoods = -(y - np.dot(X, posterior_samples.T)) ** 2 / (2 * regressor.alpha_)

# # Compute pointwise log-sum-exp
# pointwise_logsumexp = np.log(np.mean(np.exp(log_likelihoods), axis=1))

# # Compute WAIC
# waic = -2 * (np.sum(pointwise_logsumexp) - np.sum(np.var(log_likelihoods, axis=1)))
# print("WAIC:", waic)

## Calculate a weak form of each canidate

In [18]:
acc=4
# Define the derivative:
d_dx = FinDiff(0, dx, 1, acc=acc)
d_dxx = FinDiff(0, dx, 2, acc=acc)
d_dxxxx = FinDiff(0, dx, 4, acc=acc)
d_dt = FinDiff(1, dt, 1, acc=acc)

coeff = np.linalg.lstsq(np.stack([u*d_dx(u), d_dxx(u), d_dxxxx(u)]).reshape(3, -1).T, 
                        d_dt(u).reshape(-1,1), rcond=None)[0]
print((100*np.abs(coeff+1)).mean())
coeff

0.01072770464374632


array([[-0.99983724],
       [-0.99991587],
       [-0.99992506]])

In [19]:
Xf = np.stack([integrate.trapz(d_dxx(u), t, dt, axis=1), 
               integrate.trapz(d_dxxxx(u), t, dt, axis=1), 
               integrate.trapz(u*d_dx(u), t, dt, axis=1)]).T
yf = integrate.trapz(d_dt(u), t, dt, axis=1).reshape(-1,1)
coeff = np.linalg.lstsq(Xf, yf, rcond=None)[0]
print((100*np.abs(coeff+1)).mean()) # 0.00867587799540844
coeff

0.00867587799540844


array([[-1.00009514],
       [-1.00010748],
       [-1.00005765]])

In [20]:
# sio.savemat("../../PDE_Discovery_Weak_Formulation/Datasets/KS_data_tmp.mat", 
#             {'uu': un, 'xx':x, 'tt':t, 'dx':dx, 'dt': dt})

In [21]:
def weight_poly(x, m, k):
    """
    Polynomial piece of weighting function used to satisfy BC A = d^k/dx^k[(x^2 - 1)^m]
    x: independent variable
    m: power of base function
    k: order of derivative
    """
    a = np.zeros((2*m+1, 1)) # initial coefficient vector
    for l in range(m+1):
        a[2*l] = ((-1)**(m-l))*np.math.comb(m, l) # set polynomial coefficients
    
    c = np.zeros((2*m+1, 1)) # final coefficient vector
    for n in range(2*m-k+1):
        c[n] = a[n+k]*np.math.factorial(n+k)/np.math.factorial(n)
    
    p = 0
    for n in range(2*m-k+1):
        p += c[n]*(x**n) # final windowing function

    return p

In [22]:
def weight_full(k, p, x, t):
    """
    Assemble the 1D weight functions into the full weight
    k = [kx,ky,kt]: order of derivative(s)
    p = [px,py,pt]: exponents of weight polynomials
    """
    if len(k) == 3:
        wx = weight_poly(x, p[0], k[0])
        wy = weight_poly(x, p[1], k[1])
        wt = weight_poly(t, p[2], k[2])
        wX, wY, wT = np.meshgrid(wx, wy, wt)
        W = wX * wY * wT
    elif len(k) == 2:
        wx = weight_poly(x, p[0], k[0])
        wt = weight_poly(t, p[1], k[1])
        wT, wX = np.meshgrid(wt, wx)
        W = wX * wT
    
    return W, wx, wt

In [23]:
u_weak = un.copy()
n_domain = 10000
# x_size, t_size = 2**7, 2**7
x_size, t_size = 100, 100

xsup = np.linspace(-1,1,x_size+1)
tsup = np.linspace(-1,1,t_size+1)
S_x = 2/(dx*x_size)
S_t = 2/(dt*t_size)
dx, dt = xsup[1]-xsup[0], tsup[1]-tsup[0]

p = [4,3]
dA01, _, _ = weight_full([0,1], p, xsup, tsup)
dA10, _, _ = weight_full([1,0], p, xsup, tsup)
dA20, _, _ = weight_full([2,0], p, xsup, tsup)
dA40, _, _ = weight_full([4,0], p, xsup, tsup)

In [24]:
X_weak = []
y_weak = []
# X_weak_romb = []
# y_weak_romb = []
for _ in range(n_domain):
    ix = np.random.randint(0, len(x)-x_size, size=1)[0] + np.arange(0, x_size+1)
    it = np.random.randint(0, len(t)-t_size, size=1)[0] + np.arange(0, t_size+1)
    
    usup = u_weak[ix, :][:, it]
    
    f = -(1/2)*(usup**2)*dA10*S_x
    f1 = trapz(trapz(f, tsup, dt, 1), xsup, dx, 0)
    # f11 = romb(romb(f, dt, 1), dx, 0)
    
    f = usup*dA20*(S_x**2)
    f2 = trapz(trapz(f, tsup, dt, 1), xsup, dx, 0)
    # f22 = romb(romb(f, dt, 1), dx, 0)
    
    f = usup*dA40*(S_x**4)
    f3 = trapz(trapz(f, tsup, dt, 1), xsup, dx, 0)
    # f33 = romb(romb(f, dt, 1), dx, 0)
    
    X_weak.append([f1, f2, f3])
    # X_weak_romb.append([f11, f22, f33])
    
    f = -usup*dA01*S_t
    f4 = trapz(trapz(f, tsup, dt, 1), xsup, dx, 0)
    # f44 = romb(romb(f, dt, 1), dx, 0)
    
    y_weak.append(f4)
    # y_weak_romb.append(f44)
    
X_weak = np.array(X_weak)
y_weak = np.array(y_weak)

coeff = np.linalg.lstsq(X_weak, y_weak, rcond=None)[0]
(coeff, (np.abs(coeff+1)*100).mean())

# X_weak_romb = np.array(X_weak_romb)
# y_weak_romb = np.array(y_weak_romb)
# coeff = np.linalg.lstsq(X_weak_romb, y_weak_romb, rcond=None)[0]
# coeff, (np.abs(coeff+1)*100).mean()

(array([-0.98819743, -0.98149483, -0.98205128]), 1.6085484422503349)

In [25]:
p, q = 5, 4
wx, wt = np.meshgrid((xsup**2-1)**p, (tsup**2-1)**q)
W = wx*wt

# Define the derivative:
d_dx = FinDiff(0, xsup[1]-xsup[0], 1)
d_dxx = FinDiff(0, xsup[1]-xsup[0], 2)
d_dxxxx = FinDiff(0, xsup[1]-xsup[0], 4)
d_dt = FinDiff(1, tsup[1]-tsup[0], 1)

dA01 = d_dt(W)
dA10 = d_dx(W)
dA20 = d_dxx(W)
dA40 = d_dxxxx(W)

#### Denoised weak form (Savgol V1)

In [26]:
# min_loss = 1e6; best_coeff = None
# for ws in trange(3, 33, 2):
#     X_weak = []
#     y_weak = []
#     denoise_u_weak = savgol_filter(u_weak, ws, 2)
#     for _ in range(n_domain):
#         ix = np.random.randint(0, len(x)-x_size, size=1)[0] + np.arange(0, x_size+1)
#         it = np.random.randint(0, len(t)-t_size, size=1)[0] + np.arange(0, t_size+1)

#         usup = denoise_u_weak[ix, :][:, it]

#         f1 = -(1/2)*(usup**2)*dA10*S_x
#         f1 = trapz(trapz(f1, tsup, 1), xsup, 0)

#         f2 = usup*dA20*(S_x**2)
#         f2 = trapz(trapz(f2, tsup, 1), xsup, 0)

#         f3 = usup*dA40*(S_x**4)
#         f3 = trapz(trapz(f3, tsup, 1), xsup, 0)

#         X_weak.append([f1, f2, f3])

#         f4 = -usup*dA01*S_t
#         f4 = trapz(trapz(f4, tsup, 1), xsup, 0)
#         y_weak.append(f4)

#     X_weak = np.array(X_weak)
#     y_weak = np.array(y_weak)
#     coeff = np.linalg.lstsq(X_weak, y_weak, rcond=None)[0]
    
#     loss = ((X_weak@coeff-y_weak)**2).mean()
#     if loss < min_loss:
#         min_loss = loss
#         best_coeff = coeff
       
# min_loss, best_coeff, (np.abs(best_coeff+1)*100).mean() # Nice result (0.2839937497863376)

#### Denoised weak form (Savgol V2)

In [27]:
min_loss = 1e6; best_coeff = None
for ws in trange(3, 33, 2):
    X_weak = []
    y_weak = []
    for _ in range(n_domain):
        ix = np.random.randint(0, len(x)-x_size, size=1)[0] + np.arange(0, x_size+1)
        it = np.random.randint(0, len(t)-t_size, size=1)[0] + np.arange(0, t_size+1)
        
        usup = savgol_filter(u_weak[ix, :][:, it], ws, 2)

        f1 = -(1/2)*(usup**2)*dA10*S_x
        f1 = trapz(trapz(f1, tsup, dt, 1), xsup, dx, 0)
#         f1 = romb(romb(f1, dt, 1), dx, 0)

        f2 = usup*dA20*(S_x**2)
        f2 = trapz(trapz(f2, tsup, dt, 1), xsup, dx, 0)
#         f2 = romb(romb(f2, dt, 1), dx, 0)

        f3 = usup*dA40*(S_x**4)
        f3 = trapz(trapz(f3, tsup, dt, 1), xsup, dx, 0)
#         f3 = romb(romb(f3, dt, 1), dx, 0)

        X_weak.append([f1, f2, f3])

        f4 = -usup*dA01*S_t
        f4 = trapz(trapz(f4, tsup, dt, 1), xsup, dx, 0)
#         f4 = romb(romb(f4, dt, 1), dx, 0)
        y_weak.append(f4)

    X_weak = np.array(X_weak)
    y_weak = np.array(y_weak)
    coeff = np.linalg.lstsq(X_weak, y_weak, rcond=None)[0]
    
    loss = ((X_weak@coeff-y_weak)**2).mean()
    if loss < min_loss:
        min_loss = loss
        best_coeff = coeff

# Maybe do simulation next?
min_loss, best_coeff, (np.abs(best_coeff+1)*100).mean() # Nice result (0.28399369505097694)

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
100%|████████████████████████████████████████████████████████████████████| 15/15 [02:17<00:00,  9.18s/it]


(2.770655590726532e-06,
 array([-0.99455246, -0.99726875, -0.9999125 ]),
 0.27554316548480395)

In [28]:
step = 0.01
hyperparameter_val_range = np.arange(step, 1, step)
hyperparameter_val_range

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [29]:
min_loss = 1e6; best_coeff = None
for hyperparameter_val in tqdm(hyperparameter_val_range):
    X_weak = []
    y_weak = []
    denoise_u_weak = filtfilt(*butter(2, hyperparameter_val), u_weak)
    for _ in range(n_domain):
        ix = np.random.randint(0, len(x)-x_size, size=1)[0] + np.arange(0, x_size+1)
        it = np.random.randint(0, len(t)-t_size, size=1)[0] + np.arange(0, t_size+1)
        
        usup = denoise_u_weak[ix, :][:, it]

        f1 = -(1/2)*(usup**2)*dA10*S_x
        f1 = trapz(trapz(f1, tsup, dt, 1), xsup, dx, 0)
#         f1 = romb(romb(f1, dt, 1), dx, 0)

        f2 = usup*dA20*(S_x**2)
        f2 = trapz(trapz(f2, tsup, dt, 1), xsup, dx, 0)
#         f2 = romb(romb(f2, dt, 1), dx, 0)

        f3 = usup*dA40*(S_x**4)
        f3 = trapz(trapz(f3, tsup, dt, 1), xsup, dx, 0)
#         f3 = romb(romb(f3, dt, 1), dx, 0)

        X_weak.append([f1, f2, f3])

        f4 = -usup*dA01*S_t
        f4 = trapz(trapz(f4, tsup, dt, 1), xsup, dx, 0)
#         f4 = romb(romb(f4, dt, 1), dx, 0)
        y_weak.append(f4)

    X_weak = np.array(X_weak)
    y_weak = np.array(y_weak)
    coeff = np.linalg.lstsq(X_weak, y_weak, rcond=None)[0]
    
    loss = ((X_weak@coeff-y_weak)**2).mean()
    if loss < min_loss:
        min_loss = loss
        best_coeff = coeff

# Maybe do simulation next?
min_loss, best_coeff, (np.abs(best_coeff+1)*100).mean() # Nice result (0.31068650078021803)

100%|████████████████████████████████████████████████████████████████████| 99/99 [07:32<00:00,  4.57s/it]


(2.7701685183580024e-06,
 array([-0.9944473 , -0.99703951, -0.99960276]),
 0.2970143422940182)

#### Denoised weak form (Wiener, Current best)

In [30]:
min_loss = 1e6; best_coeff = None
for ws in trange(3, 33, 1):
    X_weak = []
    y_weak = []
    
    for _ in range(n_domain):
        ix = np.random.randint(0, len(x)-x_size, size=1)[0] + np.arange(0, x_size+1)
        it = np.random.randint(0, len(t)-t_size, size=1)[0] + np.arange(0, t_size+1)
        
        usup = wiener(u_weak[ix, :][:, it], ws)

        f1 = -(1/2)*(usup**2)*dA10*S_x
        f1 = trapz(trapz(f1, tsup, dt, 1), xsup, dx, 0)
#         f1 = romb(romb(f1, dt, 1), dx, 0)

        f2 = usup*dA20*(S_x**2)
        f2 = trapz(trapz(f2, tsup, dt, 1), xsup, dx, 0)
#         f2 = romb(romb(f2, dt, 1), dx, 0)

        f3 = usup*dA40*(S_x**4)
        f3 = trapz(trapz(f3, tsup, dt, 1), xsup, dx, 0)
#         f3 = romb(romb(f3, dt, 1), dx, 0)

        X_weak.append([f1, f2, f3])

        f4 = -usup*dA01*S_t
        f4 = trapz(trapz(f4, tsup, dt, 1), xsup, dx, 0)
#         f4 = romb(romb(f4, dt, 1), dx, 0)
        y_weak.append(f4)

    X_weak = np.array(X_weak)
    y_weak = np.array(y_weak)
    coeff = np.linalg.lstsq(X_weak, y_weak, rcond=None)[0]
    
    loss = ((X_weak@coeff-y_weak)**2).mean()
    if loss < min_loss:
        min_loss = loss
        best_coeff = coeff

# Maybe do simulation next?
# Nice result (0.25734593832127023, 0.21403537689738497)
min_loss, best_coeff, (np.abs(best_coeff+1)*100).mean()

100%|████████████████████████████████████████████████████████████████████| 30/30 [07:31<00:00, 15.05s/it]


(3.001980916828565e-06,
 array([-1.00137709, -0.99922417, -1.00434196]),
 0.2164960913560813)